In [3]:
import random
import random
from nltk.corpus import words

class Player:
    def __init__(self, name):
        self.name = name
        self.rack = []

    def __str__(self):
        return self.name

class Scrabble:
    def __init__(self):
        self.board = [[' ' for _ in range(15)] for _ in range(15)]
        self.players = []
        self.scores = []
        self.bag = []
        self.special_tiles_used = []
        self.initialize_bag()
        self.initialize_players()
        self.initialize_board()

    def initialize_bag(self):
        # Create a bag of tiles with letter and score attributes
        tile_distribution = {
            'A': (9, 1), 'B': (2, 3), 'C': (2, 3), 'D': (4, 2), 'E': (12, 1), 'F': (2, 4), 'G': (3, 2),
            'H': (2, 4), 'I': (9, 1), 'J': (1, 8), 'K': (1, 5), 'L': (4, 1), 'M': (2, 3), 'N': (6, 1),
            'O': (8, 1), 'P': (2, 3), 'Q': (1, 10), 'R': (6, 1), 'S': (4, 1), 'T': (6, 1), 'U': (4, 1),
            'V': (2, 4), 'W': (2, 4), 'X': (1, 8), 'Y': (2, 4), 'Z': (1, 10), '_': (2, 0)  # Blank tile represented as ''
        }

        # Populate the bag with tiles based on their distribution
        for tile, (count, score) in tile_distribution.items():
            self.bag.extend([{'letter': tile, 'score': score} for _ in range(count)])

        random.shuffle(self.bag)

    def initialize_players(self):
        num_players = int(input("Enter the number of players: "))
        for i in range(num_players):
            name = input(f"Enter the name of Player {i + 1}: ")
            player = Player(name)
            self.players.append(player)
            self.scores.append(0)


    def initialize_board(self):
        # Initialize a blank board
        # Set up the starting positions for special tiles like Double Word Score or Triple Letter Score
        # Let's assume the starting positions for special tiles are fixed
        special_tiles_DW = [(1, 1), (1, 13), (2, 2), (2, 12), (3, 3), (3, 11), (4, 4), (4, 10), (10, 4), 
         (10, 10), (11, 3), (11, 11), (12, 2), (12, 12), (13, 1), (13, 13)]

        special_titles_TW = [(0, 0), (0, 7), (0, 14), (7, 0), (7, 14), (14, 0), (14, 7), (14, 14)]

        double_letter_scores = [(0, 3), (0, 11), (2, 6), (2, 8), (3, 0), (3, 7), (3, 14), (6, 2), 
         (6, 6), (6, 8), (6, 12), (7, 3), (7, 11), (8, 2), (8, 6), (8, 8), (8, 12), (11, 0), (11, 7), (11, 14), (12, 6), (12, 8), (14, 3), (14, 11)]

        triple_letter_scores = [(1, 5), (1, 9), (5, 1), (5, 5), (5, 9), (5, 13), (9, 1), (9, 5), (9, 9), (9, 13), (13, 5), (13, 9)]

        # for row, col in special_tiles_DW:
        #     self.board[row][col] = '*'

        # for row, col in special_titles_TW:
        #     self.board[row][col] = '#'

        # for row, col in double_letter_scores:
        #     self.board[row][col] = '%'

        # for row, col in triple_letter_scores:
        #     self.board[row][col] = '&'

    def play_game(self):
        game_over = False
        current_player = 0
        rack = self.draw_tiles(current_player, [])

        while not game_over:
            player_name = self.players[current_player]
            
            print(f"\n{player_name}'s turn:")
            print(f"Current score: {self.scores[current_player]}")
            print(f"Tiles in hand: {self.tiles_to_str(rack)}")

            self.display_board()

            word = input("Enter the word to play: ")
            word = word.upper()

            row, col = self.get_starting_position()
            direction = input("Enter the direction (across/down): ").lower()

            if self.is_valid_move(word, row, col, direction, rack):
                score = self.calculate_score(word, row, col, direction)
                self.update_board(word, row, col, direction)
                self.update_score(current_player, score)

                if len(self.bag) == 0 and len(rack) == 0:
                    game_over = True
                    print("No more tiles left. Game over!")
                else:
                    current_player = (current_player + 1) % len(self.players)
                    rack = self.draw_tiles(current_player, rack)  # Draw tiles for the next player
            else:
                print("Invalid move. Please try again.")
    
    def update_rack(self, player_index, tiles_used):
        rack = self.players[player_index].rack

        for tile_used in tiles_used:
            for tile in rack:
                if tile['letter'] == tile_used['letter']:
                    rack.remove(tile)
                    break

        tiles_needed = 7 - len(rack)
        for _ in range(tiles_needed):
            if len(self.bag) > 0:
                new_tile = self.bag.pop()
                rack.append(new_tile)
            else:
                break


    def draw_tiles(self, player_index, used_tiles):
        rack = self.players[player_index].rack

        # Remove used tiles from the rack
        updated_rack = []
        for tile in rack:
            if tile not in used_tiles:
                updated_rack.append(tile)
        rack = updated_rack

        # Fill the rack up to 7 tiles
        tiles_needed = 7 - len(rack)
        for _ in range(tiles_needed):
            if len(self.bag) > 0:
                new_tile = self.bag.pop()
                rack.append(new_tile)
            else:
                break

        return rack

    def is_valid_move(self, word, row, col, direction, rack):
        rack_letters = [tile['letter'] for tile in rack]
        
        if word.lower() not in words.words():
          print("Invalid move. The word is not in the dictionary.")
          return False
        
        else:
          if direction == 'across':
              for i in range(len(word)):
                  if self.board[row][col + i] != ' ' and self.board[row][col + i] != word[i]:
                      return False
                  if self.board[row][col + i] == ' ':
                      if word[i] not in rack_letters and '_' not in rack_letters:
                          return False
                      if word[i] in rack_letters:
                          rack_letters.remove(word[i])
                      elif '_' in rack_letters:
                          rack_letters.remove('_')
          elif direction == 'down':
              for i in range(len(word)):
                  if self.board[row + i][col] != ' ' and self.board[row + i][col] != word[i]:
                      return False
                  if self.board[row + i][col] == ' ':
                      if word[i] not in rack_letters and '_' not in rack_letters:
                          return False
                      if word[i] in rack_letters:
                          rack_letters.remove(word[i])
                      elif '_' in rack_letters:
                          rack_letters.remove('_')
          else:
              return False

          return True

    def calculate_score(self, word, row, col, direction):
        score = 0
        word_multiplier = 1

        special_tiles_DW = [(1, 1), (1, 13), (2, 2), (2, 12), (3, 3), (3, 11), (4, 4), (4, 10), (10, 4), 
         (10, 10), (11, 3), (11, 11), (12, 2), (12, 12), (13, 1), (13, 13)]

        special_titles_TW = [(0, 0), (0, 7), (0, 14), (7, 0), (7, 14), (14, 0), (14, 7), (14, 14)]

        double_letter_scores = [(0, 3), (0, 11), (2, 6), (2, 8), (3, 0), (3, 7), (3, 14), (6, 2), 
         (6, 6), (6, 8), (6, 12), (7, 3), (7, 11), (8, 2), (8, 6), (8, 8), (8, 12), (11, 0), (11, 7), (11, 14), (12, 6), (12, 8), (14, 3), (14, 11)]

        triple_letter_scores = [(1, 5), (1, 9), (5, 1), (5, 5), (5, 9), (5, 13), (9, 1), (9, 5), (9, 9), (9, 13), (13, 5), (13, 9)]

        for letter in word:
            letter_score = self.get_tile_score(letter)

            if direction == 'across':
                if (row, col) not in self.special_tiles_used:
                    if (row, col) in double_letter_scores:
                        letter_score *= 2
                        self.special_tiles_used.append((row, col))
                    elif (row, col) in triple_letter_scores:
                        letter_score *= 3
                        self.special_tiles_used.append((row, col))
                    elif (row, col) in special_tiles_DW:
                        word_multiplier *= 2
                        self.special_tiles_used.append((row, col))
                    elif (row, col) in special_titles_TW:
                        word_multiplier *= 3
                        self.special_tiles_used.append((row, col))

                col += 1  # move to the next column
            else:  # 'down'
                if (row, col) not in self.special_tiles_used:
                    if (row, col) in double_letter_scores:
                        letter_score *= 2
                        self.special_tiles_used.append((row, col))
                    elif (row, col) in triple_letter_scores:
                        letter_score *= 3
                        self.special_tiles_used.append((row, col))
                    elif (row, col) in special_tiles_DW:
                        word_multiplier *= 2
                        self.special_tiles_used.append((row, col))
                    elif (row, col) in special_titles_TW:
                        word_multiplier *= 3
                        self.special_tiles_used.append((row, col))

                row += 1  # move to the next row

            score += letter_score

        score *= word_multiplier
        return score

    def update_board(self, word, row, col, direction):
        if direction == 'across':
            for i in range(len(word)):
                self.board[row][col + i] = word[i]
        elif direction == 'down':
            for i in range(len(word)):
                self.board[row + i][col] = word[i]

    def update_score(self, player_index, score):
        self.scores[player_index] += score

    def get_tile_score(self, letter):
        # Return the score of a tile based on its letter
        tile_scores = {
            'A': 1, 'B': 3, 'C': 3, 'D': 2, 'E': 1, 'F': 4, 'G': 2,
            'H': 4, 'I': 1, 'J': 8, 'K': 5, 'L': 1, 'M': 3, 'N': 1,
            'O': 1, 'P': 3, 'Q': 10, 'R': 1, 'S': 1, 'T': 1, 'U': 1,
            'V': 4, 'W': 4, 'X': 8, 'Y': 4, 'Z': 10, '_': 0
        }
        return tile_scores[letter]

    def get_starting_position(self):
        row = int(input("Enter the starting row (0-14): "))
        col = int(input("Enter the starting column (0-14): "))
        return row, col

    def display_board(self):
        print()
        print('    ' + ' '.join([str(i) for i in range(15)]))
        print()
        for i in range(15):
            row_str = str(i).rjust(2) + '  '
            for j in range(15):
                row_str += self.board[i][j] + ' '
            print(row_str)
        print()

    def display_scores(self):
        print("Current Scores:")
        for player, score in zip(self.players, self.scores):
            print(f"{player}: {score}")
        print()

    def tiles_to_str(self, tiles):
        return ', '.join([f"{tile['letter']} ({tile['score']})" for tile in tiles])

if __name__ == "__main__":
    game = Scrabble()
    game.play_game()


trenty's turn:
Current score: 0
Tiles in hand: S (1), Y (4), A (1), G (2), S (1), N (1), L (1)

    0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

 0                                
 1                                
 2                                
 3                                
 4                                
 5                                
 6                                
 7                                
 8                                
 9                                
10                                
11                                
12                                
13                                
14                                


b's turn:
Current score: 0
Tiles in hand: E (1), P (3), E (1), O (1), D (2), Y (4), L (1)

    0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

 0                                
 1                                
 2                                
 3                                
 4                                
 5                            

ValueError: invalid literal for int() with base 10: ''

In [ ]:
    0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

 0  #     %       #       %     # 
 1    *       &       &       *   
 2      *       %   %       *     
 3  %     *       %       *     % 
 4          *           *         
 5    &       &       &       &   
 6      %       %   %       %     
 7  #     %               %     # 
 8      %       %   %       %     
 9    &       &       &       &   
10          *           *         
11  %     *       %       *     % 
12      *       %   %       *     
13    *       &       &       *   
14  #     %       #       %     # 

    0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

 0  #     %       #       %     # 
 1    *       &       &       *   
 2      *       %   %       *     
 3  %     *       %       *     % 
 4          *           *         
 5    &       &       &       &   
 6      %       %   %       %     
 7  #     %               %     # 
 8      %       %   %       %     
 9    &       &       &       &   
10          *           *         
11  %     *       %       *     % 
12      *       %   %       *     
13    *       &       &       *   
14  #     %       #       %     # 

Issues to overcome:

1. The new word should not count double or triple points once that tile is used before.
2. The tiles in hand should shuffle only the letters that were used in the previous turn by the same player.
3. Add single letter swap letter option to each turn and turn gets over.
4. The first word should overlap on 7,7 tile.